## **1.7_DATASET_masters_remove_atypical_genotype_calls.ipynb**

### GOALS of this script:
* part two of whole merging workflow for the masters
* numbering in accordance to workflow_europe_maize_dataset_prep_masters
* checked the columns of the 600k array info file
* combination/check of:
    * merging_script_final-Copy1.ipynb
    * merging_script_final.ipynb
    * the corresponding number steps
* OUT: filtered_xxx_for_masters.csv

### *Import packages*

In [ ]:
import gzip
import allel
import pandas as pd
import numpy as np
import tskit
import tsinfer
import sys
import json
import csv
from IPython.display import SVG
from IPython.display import HTML
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import Workbook
from progressbar import ProgressBar
sns.set_style('white')
sns.set_style('ticks')

### *Import files*

In [ ]:
final_columns_unterseer_2016_elite=pd.read_csv("final_columns_unterseer_2016_elite_for_masters.csv",sep="\t")

In [ ]:
final_columns_unterseer_2016_elite

In [ ]:
final_columns_unterseer_2016_landraces=pd.read_csv("final_columns_unterseer_2016_landraces_for_masters.csv",sep="\t")

In [ ]:
final_columns_unterseer_2016_landraces

In [ ]:
final_columns_mayer_2017_landraces=pd.read_csv("final_columns_mayer_2017_landraces_for_masters.csv",sep="\t")

In [ ]:
final_columns_mayer_2017_landraces

In [ ]:
final_columns_mayer_2020_dhlines=pd.read_csv("final_columns_mayer_2020_dhlines_for_masters.csv",sep="\t")

In [ ]:
final_columns_mayer_2020_dhlines

### *1.7.1 Get an array with markers from array.txt which are associated with a-typical genotype calls*

In [ ]:
#read-in the array.txt
check_strand=pd.read_csv("/Users/kschul38/Documents/tsinfer-project/data/2_processed/europe_maize_dataset_600k/600k_array_info_header_removed.txt", sep="\t", dtype = str)
check_strand

**Print the header for the 600k array info file**

In [ ]:
N=16

In [ ]:
with open("/Users/kschul38/Documents/tsinfer-project/data/1_raw/600k_array_info_raw.txt") as myfile:
    header = [next(myfile) for x in range(N)]

In [ ]:
header

**Check the columns**

In [ ]:
#Counter(check_strand["ID"]==check_strand["probe_set_ID"])

In [ ]:
#Counter(check_strand["organism"])

In [ ]:
#Counter(check_strand["genome_acc"])

In [ ]:
#Counter(check_strand["chromosome"])

In [ ]:
Counter(check_strand["allele_A"])

In [ ]:
Counter(check_strand["allele_B"])

#### *Create boolean to filter the markers*

In [ ]:
#get all the genotype-calls from the array.txt
genotype_call=check_strand["alleles_f"].values.ravel()

In [ ]:
#check that number of genotype calls is correct
len(genotype_call)

In [ ]:
#get a list of all unique calls
unique_values_array = pd.unique(genotype_call)

In [ ]:
#unique_values_array

In [ ]:
#look at complete list
#unique_values_array

In [ ]:
#remove the "typical genotype calls" - so that just indels remain
unique_values_array = np.delete(unique_values_array, [0,1,2,3,4,5])

In [ ]:
#check that it only contains a-typical genotype calls
#unique_values_array

In [ ]:
#create a boolean over the whole allele column, where typical calls == True & atypical calls == False 
array_marker_filter_table=~check_strand["alleles_f"].isin(unique_values_array)

In [ ]:
array_marker_filter_table

In [ ]:
Counter(array_marker_filter_table)

#### *Apply the boolean to the array-marker column*

In [ ]:
#get all marker IDs from the array.txt
array_non_typical_genotype_calls_marker=check_strand['ID']

In [ ]:
#list of all marker IDs
#array_non_typical_genotype_calls_marker

In [ ]:
#convert to array
array_non_typical_genotype_calls_marker=array_non_typical_genotype_calls_marker.to_numpy()

In [ ]:
#array of all marker IDs
#array_non_typical_genotype_calls_marker

In [ ]:
#filter marker array using the inversion of the boolean (TRUE=atypical, FALSE=typical)
array_non_typical_genotype_calls_marker=array_non_typical_genotype_calls_marker[~array_marker_filter_table]

In [ ]:
#list of marker IDs that belong to the a-typical genotype calls
array_non_typical_genotype_calls_marker

In [ ]:
#check length
len(array_non_typical_genotype_calls_marker)

**Reapply the boolean to the 600k array info txt**

In [ ]:
filter_array_600k=~check_strand["ID"].isin(array_non_typical_genotype_calls_marker)

In [ ]:
Counter(filter_array_600k)

In [ ]:
filtered_array_600k=check_strand[filter_array_600k]

In [ ]:
filtered_array_600k

In [ ]:
Counter(filtered_array_600k["alleles_f"])

In [ ]:
Counter(filtered_array_600k["allele_A"])

In [ ]:
Counter(filtered_array_600k["allele_B"])

### *1.7.2 Filter the dataset - Unterseer_elite_2016*

*Create boolean based on Unterseer_elite_2016*

In [ ]:
#615884 row number of the table before filtering

In [ ]:
#filter out the a-typical genotype calls by first filering the for isin and then using the inverse
filter_boolean_unterseer_elite_2016=~final_columns_unterseer_2016_elite["variants/AD_2016_elite"].isin(array_non_typical_genotype_calls_marker)

In [ ]:
Counter(filter_boolean_unterseer_elite_2016)

In [ ]:
#615884-6752=609132 row number of the table after filtering

In [ ]:
#apply the filter to the unfiltered table
filtered_unterseer_2016_elite_right_pos=final_columns_unterseer_2016_elite[filter_boolean_unterseer_elite_2016]

In [ ]:
#the filtered table
#filtered_unterseer_2016_elite_right_pos

### *1.7.3 Filter the dataset - Unterseer_landraces_2016*

*Create boolean based on Unterseer_landraces_2016*

In [ ]:
#615884 row number of the table before filtering

In [ ]:
#filter out the a-typical genotype calls by first filering the for isin and then using the inverse
filter_boolean_unterseer_landraces_2016=~final_columns_unterseer_2016_landraces["variants/AD_2016_landraces"].isin(array_non_typical_genotype_calls_marker)

In [ ]:
Counter(filter_boolean_unterseer_landraces_2016)

In [ ]:
#615884-6752=609132 row number of the table after filtering

In [ ]:
#apply the filter to the unfiltered table
filtered_unterseer_2016_landraces_right_pos=final_columns_unterseer_2016_landraces[filter_boolean_unterseer_landraces_2016]

In [ ]:
#the filtered table
#filtered_unterseer_2016_landraces_right_pos

### *1.7.4 Filter the dataset - Mayer_2017_landraces*

*Create boolean based on Mayer_2017_landraces*

In [ ]:
#616201 row number of the table before filtering

In [ ]:
#filter out the a-typical genotype calls by first filering the for isin and then using the inverse
filter_boolean_mayer_2017_landraces_suffix=~final_columns_mayer_2017_landraces["index_2017_landraces"].isin(array_non_typical_genotype_calls_marker)

In [ ]:
Counter(filter_boolean_mayer_2017_landraces_suffix)

In [ ]:
#616201-6759=609442 row number of the table after filtering

In [ ]:
#apply the filter to the unfiltered table
filtered_mayer_2017_landraces_suffix=final_columns_mayer_2017_landraces[filter_boolean_mayer_2017_landraces_suffix]

In [ ]:
#the filtered table
#filtered_mayer_2017_landraces_suffix

### *1.7.5 Filter the dataset - Mayer_2020_dhlines*

*Create boolean based on Mayer_2020_dhlines*

In [ ]:
#616201 row number of the table before filtering

In [ ]:
#filter out the a-typical genotype calls by first filering the for isin and then using the inverse
filter_boolean_mayer_2020_dhlines_suffix=~final_columns_mayer_2020_dhlines["marker_2020_dh"].isin(array_non_typical_genotype_calls_marker)

In [ ]:
Counter(filter_boolean_mayer_2020_dhlines_suffix)

In [ ]:
#616201-6759=609442 row number of the table after filtering

In [ ]:
#apply the filter to the unfiltered table
filtered_mayer_2020_dhlines_suffix=final_columns_mayer_2020_dhlines[filter_boolean_mayer_2020_dhlines_suffix]

In [ ]:
#the filtered table
filtered_mayer_2020_dhlines_suffix

### *1.7.6 Check if tables now only contain typical genotype calls*

**Unterseer_2016 elite**

In [ ]:
#Counter(final_columns_unterseer_2016_elite["alleles_2016_elite"])

In [ ]:
Counter(filtered_unterseer_2016_elite_right_pos["alleles_2016_elite"])

**Unterseer_2016 landraces**

In [ ]:
#Counter(final_columns_unterseer_2016_landraces["alleles_2016_landraces"])

In [ ]:
Counter(filtered_unterseer_2016_landraces_right_pos["alleles_2016_landraces"])

**Mayer_2017 landraces**

In [ ]:
mayer_2017_landraces_get_alleles_before_filter=final_columns_mayer_2017_landraces.drop(['index_2017_landraces','chr_v4_2017_landraces','pos_v4_2017_landraces','quality_2017_landraces'], axis = 1)

In [ ]:
#mayer_2017_landraces_get_alleles_before_filter

In [ ]:
mayer_2017_all_geno_values_before_filter = mayer_2017_landraces_get_alleles_before_filter.values.ravel()

In [ ]:
unique_values_mayer_2017_before_filter = pd.unique(mayer_2017_all_geno_values_before_filter)

In [ ]:
unique_values_mayer_2017_before_filter

*Filtered*

In [ ]:
mayer_2017_landraces_get_alleles=filtered_mayer_2017_landraces_suffix.drop(['index_2017_landraces','chr_v4_2017_landraces','pos_v4_2017_landraces','quality_2017_landraces'], axis = 1)

In [ ]:
#mayer_2017_landraces_get_alleles

In [ ]:
mayer_2017_all_geno_values = mayer_2017_landraces_get_alleles.values.ravel()

In [ ]:
unique_values_mayer_2017 = pd.unique(mayer_2017_all_geno_values)

In [ ]:
unique_values_mayer_2017 

**Mayer_2020 dhlines**

In [ ]:
mayer_2020_dhlines_get_alleles_before_filter=final_columns_mayer_2020_dhlines.drop(['marker_2020_dh','chr_2020_dh','pos_2020_dh','quality_2020_dh'], axis = 1)

In [ ]:
mayer_2020_dhlines_get_alleles_before_filter

In [ ]:
mayer_2020_all_geno_values_before_filter = mayer_2020_dhlines_get_alleles_before_filter.values.ravel()

In [ ]:
unique_values_mayer_2020_before_filter = pd.unique(mayer_2020_all_geno_values_before_filter)

In [ ]:
unique_values_mayer_2020_before_filter

*Filtered*

In [ ]:
mayer_2020_dhlines_get_alleles=filtered_mayer_2020_dhlines_suffix.drop(['marker_2020_dh','chr_2020_dh','pos_2020_dh','quality_2020_dh'], axis = 1)

In [ ]:
mayer_2020_dhlines_get_alleles

In [ ]:
mayer_2020_all_geno_values =mayer_2020_dhlines_get_alleles.values.ravel()

In [ ]:
unique_values_mayer_2020 = pd.unique(mayer_2020_all_geno_values)

In [ ]:
unique_values_mayer_2020

## **OUTPUT**

### **Write to file**

In [ ]:
#filtered_unterseer_2016_elite_right_pos.to_csv("filtered_unterseer_2016_elite_for_masters.csv", sep="\t", index = False)

In [ ]:
#filtered_unterseer_2016_landraces_right_pos.to_csv("filtered_unterseer_2016_landraces_for_masters.csv", sep="\t", index = False)

In [ ]:
#filtered_mayer_2017_landraces_suffix.to_csv("filtered_mayer_2017_landraces_for_masters.csv",sep="\t", index = False)

In [ ]:
#filtered_mayer_2020_dhlines_suffix.to_csv("filtered_mayer_2020_dhlines_for_masters.csv",sep="\t", index = False)